<a href="https://colab.research.google.com/github/m-and-ms/Question-Clustering-/blob/master/tf_idf%2BPos_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from gensim.models import Word2Vec
import numpy as np


In [0]:
import pandas as pd

In [0]:
Q_df=pd.read_json (r'WebQSP.train.json')


In [0]:
Q_df

In [0]:
Q_df.columns

In [0]:
Q_df['Questions'][1]['ProcessedQuestion']

In [0]:
num_questions=len(Q_df['Questions'])

In [0]:
Sentences_Data=[Q_df['Questions'][idx]['ProcessedQuestion'] for idx in range(num_questions) ]

In [0]:
len(Sentences_Data)

In [0]:
Sentences_Data

In [0]:
import re , nltk, pprint

In [0]:

from nltk import *
from nltk.corpus import stopwords 
stemmer = nltk.stem.porter.PorterStemmer()

nltk.download('stopwords')

In [0]:
stop_words = set(stopwords.words('english')) 


In [0]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

In [0]:
def clean_text(sent_list):
  cleanr = re.compile('<.*?>|&([0-9]+|#[0-9]{1,6}|#x[0-]{1,6});')
  #token_sents = [print(tk)  for sent in token_sents for tk in sent ]

  token_sents = [re.sub(cleanr, '', sent)  for sent in sent_list ]
  token_sents=[re.sub("'","",sent) for sent in token_sents  ]

  token_sents=[sent.strip("'") for sent in token_sents  ]
  token_sents=[sent.lower() for sent in token_sents  ]

  token_sents =[ word_tokenize(sent)  for sent in token_sents ]  
  #token_sents = [w for word_tokens in token_sents for w in word_tokens  if not w in stop_words]  
  token_sents = [w for word_tokens in token_sents for w in word_tokens if not w in stop_words] 

  

  return token_sents


In [0]:
data_tokens=clean_text(Sentences_Data)

In [0]:
data_tokens

In [0]:
text_tagged = nltk.pos_tag(data_tokens) 

tagged_tokens = []
for word in text_tagged:
  tagged_tokens.append(word[0] + "/" + word[1])
print(tagged_tokens)

#  doc = ' '.join(new_text)
  

In [0]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer


In [0]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(tagged_tokens)

In [0]:
tfidf_transformer = TfidfTransformer()
tfidf_matrix = tfidf_transformer.fit_transform(X_train_counts)

In [0]:
tfidf_matrix.shape

In [0]:
word2tfidf = dict(zip(count_vect.get_feature_names(), tfidf_transformer.idf_,))

In [0]:
from sklearn.cluster import AgglomerativeClustering


In [0]:
cluster = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward')


In [0]:
cluster.fit(tfidf_matrix.toarray())

In [0]:
labels = cluster.labels_



In [0]:
print(labels)

In [0]:
import matplotlib.pyplot as plt
 
from sklearn.manifold import TSNE

In [0]:
model = TSNE(n_components=2, random_state=1,metric='euclidean', n_iter=15000)


In [0]:
Y=model.fit_transform(tfidf_matrix)


In [0]:
plt.scatter(Y[:, 0], Y[:, 1], c=labels)

In [0]:
for j in range(len(Sentences_Data)):    
   print ("%s %s" % (labels[j],  Sentences_Data[j]))

In [0]:
from sklearn.cluster import KMeans 
from sklearn import metrics 
from scipy.spatial.distance import cdist 

In [0]:
kmeans = KMeans(n_clusters=7, random_state=0).fit(tfidf_matrix)
kmeans.labels_
km_tf=kmeans.fit_transform(tfidf_matrix)

In [0]:
type1_occurrences = np.count_nonzero(kmeans.labels_ == 1)

In [0]:
type2_occurences=np.count_nonzero(kmeans.labels_ == 0)

In [0]:
type3_occurences=np.count_nonzero(kmeans.labels_ == 2)

In [0]:
total_occurances=len(kmeans.labels_)

In [0]:
print("Type1 percentage",(type1_occurrences/total_occurances)*100 ,"Type2 percentage ",(type2_occurences/total_occurances)*100, "Type3 percentage ",(type3_occurences/total_occurances)*100)

In [0]:
print("Type1 count",(type1_occurrences) ,"Type2 count ",(type2_occurences), "Type3 count ",(type3_occurences))

In [0]:
plt.scatter(km_tf[:,0],km_tf[:,1],c=kmeans.labels_,marker='X')

In [0]:
y_means=kmeans.predict(tfidf_matrix)

In [0]:
kmeans.cluster_centers_.shape


In [0]:
from sklearn import decomposition
pca = decomposition.PCA(n_components=7).fit(tfidf_matrix.toarray())
colors=['red','green','blue','yellow','black','orange','brown'] 
label_colors=[colors[i] for i in kmeans.labels_]
coords=pca.transform(tfidf_matrix.toarray())

plt.scatter(coords[:,0],coords[:,1],c=label_colors)

centroids=kmeans.cluster_centers_
centroidcoords=pca.transform(centroids)
plt.scatter(centroidcoords[:,0],centroidcoords[:,1],marker='X')
plt.show()

In [0]:
from sklearn.cluster import KMeans 
from sklearn import metrics 
from scipy.spatial.distance import cdist 

distortions = [] 
inertias = [] 
mapping1 = {} 
mapping2 = {} 
K = range(1,10) 
  
for k in K: 
    #Building and fitting the model 
    kmeanModel = KMeans(n_clusters=k).fit(tfidf_matrix.toarray()) 
    kmeanModel.fit(tfidf_matrix.toarray())     
      
    distortions.append(sum(np.min(cdist(tfidf_matrix.toarray(), kmeanModel.cluster_centers_, 
                      'euclidean'),axis=1)) / np.asarray(tfidf_matrix.toarray()).shape[0]) 
    inertias.append(kmeanModel.inertia_) 
  
    mapping1[k] = sum(np.min(cdist(tfidf_matrix.toarray(), kmeanModel.cluster_centers_, 
                 'euclidean'),axis=1)) / np.asarray( tfidf_matrix.toarray()).shape[0] 
    mapping2[k] = kmeanModel.inertia_ 

plt.plot(K, distortions, 'bx-') 
plt.xlabel('Values of K') 
plt.ylabel('Distortion') 
plt.title('The Elbow Method using Distortion') 
plt.show() 